##### 1. Read File

In [0]:
yellowtaxiDF = spark.read.csv("/Volumes/azuredatabricks_1405569260508774/default/yellowtaxi/yellow_tripdata_2016-01.csv", inferSchema=True, header=True)

In [0]:
# print schema of YellowTaxi Table
print(yellowtaxiDF.schema)

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('pickup_longitude', DoubleType(), True), StructField('pickup_latitude', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('dropoff_longitude', DoubleType(), True), StructField('dropoff_latitude', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True)])


In [0]:
yellowtaxiDF.coalesce(1).write.mode("overwrite").format("parquet").save("/Volumes/azuredatabricks_1405569260508774/default/yellowparquettaxi/")

In [0]:
%sql
CREATE LIVE TABLE YellowTaxi_Bronze(
  VendorID integer,
  tpep_pickup_datetime timestamp,
  tpep_dropoff_datetime timestamp,
  passenger_count integer,
  trip_distance double,
  pickup_longitude double,
  pickup_latitude double,
  RatecodeID integer,
  store_and_fwd_flag string,
  dropoff_longitude double,
  dropoff_latitude double,
  payment_type integer,
  fare_amount double,
  extra double,
  mta_tax double,
  tip_amount double,
  tolls_amount double,
  improvement_surcharge double,
  total_amount double,
  FileName STRING,
  CreatedOn TIMESTAMP
) USING DELTA PARTITIONED BY (VendorID)
AS
SELECT * , INPUT_FILE_NAME() AS FileName, CURRENT_TIMESTAMP() AS CreatedOn
FROM parquet.`/Volumes/azuredatabricks_1405569260508774/default/yellowparquettaxi/`


Name,Type
VendorID,int
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
passenger_count,int
trip_distance,double
pickup_longitude,double
pickup_latitude,double
RatecodeID,int
store_and_fwd_flag,string
dropoff_longitude,double


In [0]:
%sql
-- Generated AS keyword by Databricks to indicate that PickUpYear column is auto-computed column, its value will
-- be automatically calculated based on the expression of the tpep_pickup_datetime column.
CREATE LIVE TABLE YellowTaxi_Silver(
  VendorID integer,
  tpep_pickup_datetime timestamp,
  tpep_dropoff_datetime timestamp,
  passenger_count integer,
  trip_distance double,
  pickup_longitude double,
  pickup_latitude double,
  RatecodeID integer,
  payment_type integer,
  fare_amount double,
  total_amount double,
  PickUpYear INT GENERATED ALWAYS AS (YEAR (tpep_pickup_datetime)),
  PickUpMonth INT GENERATED ALWAYS AS (MONTH (tpep_pickup_datetime)),
  PickUpDay INT GENERATED ALWAYS AS (DAY (tpep_pickup_datetime)),
  CreatedOn TIMESTAMP,
  -- Contraint act as conditional check statements and EXPECT is the Expectation from column value
  CONSTRAINT ValidTotalAmount EXPECT(total_amount IS NOT NULL AND total_amount > 0) ON VIOLATION DROP ROW,
  CONSTRAINT ValidDistance EXPECT(trip_distance IS NOT NULL AND trip_distance > 0) ON VIOLATION DROP ROW,
  CONSTRAINT ValidPassenger EXPECT(passenger_count > 0) ON VIOLATION DROP ROW,
  CONSTRAINT ValidVendorID EXPECT(VendorID IS NOT NULL AND VendorID > 0) ON VIOLATION FAIL UPDATE

) USING DELTA PARTITIONED BY (passenger_count)
AS
SELECT VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance,  pickup_longitude, pickup_latitude, RatecodeID, payment_type, fare_amount, total_amount, CURRENT_TIMESTAMP() AS CreatedOn
FROM live.YellowTaxi_Bronze

Name,Type
VendorID,int
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
passenger_count,int
trip_distance,double
pickup_longitude,double
pickup_latitude,double
RatecodeID,int
payment_type,int
fare_amount,double


##### Create Gold Table for Loaction-wise Summary

In [0]:
%sql
CREATE LIVE TABLE YellowTaxiGold_Summary
AS
SELECT RatecodeID, passenger_count, 
count(*) AS TotalRide,
SUM(trip_distance) AS TripDistance,
SUM(total_amount) AS TotalAmount

FROM live.YellowTaxi_Silver
GROUP BY RatecodeID, passenger_count


Name,Type
RatecodeID,int
passenger_count,int
TotalRide,bigint
TripDistance,double
TotalAmount,double


##### Create Gold Table for Date-wise Summary

In [0]:
%sql
Create LIVE Table YellowTaxiGold_SummaryByDate
AS 
SELECT PickUpYear, PickUpMonth, PickUpDay,
COUNT(*) AS Total_Rides,
SUM(trip_distance) AS TotalDistance,
SUM(total_amount) AS TotalAmount
FROM live.YellowTaxi_Silver
GROUP BY PickUpYear, PickUpMonth, PickUpDay


Name,Type
PickUpYear,int
PickUpMonth,int
PickUpDay,int
Total_Rides,bigint
TotalDistance,double
TotalAmount,double


In [0]:
%sql
CREATE MATERIALIZED VIEW Yellow_TaxiGold
AS
SELECT *
FROM live.YellowTaxi_Silver

Name,Type
VendorID,int
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
passenger_count,int
trip_distance,double
pickup_longitude,double
pickup_latitude,double
RatecodeID,int
payment_type,int
fare_amount,double


In [0]:
%sql
select * from Yellow_TaxiGold

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `Yellow_TaxiGold` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [Yellow_TaxiGold], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:90)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:224)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2$adapted(CheckAnalysis.scala:197)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:287)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.sca